# AoC Day 12 Challenge. First section is grabbing the input text for the day

In [5]:
# Common Imports
import pycurl as curl
from io import BytesIO
import pandas as pd
import numpy as np
import re
import os
from functools import partial
from copy import deepcopy
from joblib import Memory
memory = Memory('/tmp/', verbose=0)
memory.clear()
import matplotlib.pyplot as plt
import sys
from tqdm import tqdm
sys.getrecursionlimit()
sys.setrecursionlimit(100_000)

[Memory(location=/tmp/joblib)]: Flushing completely the cache


In [6]:
# Get input as `input`
year = 2024
day = 12

with open('/home/alec/aoc_cookie.txt', 'r') as file:
    cookie = file.read() 

buffer = BytesIO()
c = curl.Curl()
c.setopt(curl.URL, f"https://adventofcode.com/{year}/day/{day}/input")
c.setopt(curl.COOKIE, f"session={cookie}")
c.setopt(curl.WRITEDATA, buffer)
c.perform()
c.close()

input = buffer.getvalue().decode('utf-8')

# Part 1

In [7]:
entries = input.splitlines()

In [8]:
char_arr = np.array([[char for char in entry] for entry in entries])
region_types = list(map(chr, range(65, 65 + 26)))

def add(p1, p2):
    tmp = []
    for _p1, _p2 in zip(p1, p2):
        tmp.append(_p1 + _p2)
    return tuple(tmp)

def diff(p1, p2):
    tmp = []
    for _p1, _p2 in zip(p1,p2):
        tmp.append(_p1 - _p2)
    return tuple(tmp)

def inbounds(coord):
    for c in coord:
        if c < 0 or c >= char_arr.shape[0]:
            return False
    return True

def index(coord):
    return char_arr[coord[0], coord[1]]

d_unwrap = [(-1,0), (0,1), (1,0), (0,-1)]
verts = set([0,2])
d = 0
total = 0
coord2region = {}

def find_all_region(coord, id, coord2region):
    for d in range(4):
        new_coord = add(coord, d_unwrap[d])
        if not inbounds(new_coord):
            continue
        if index(coord) == index(new_coord):
            if (new_coord[0], new_coord[1], index(coord)) not in coord2region:
                coord2region[(new_coord[0], new_coord[1], index(coord))] = id
                find_all_region(new_coord, id, coord2region)

region2char = {}

region_id = 0
# traverse the graph row x col
for row in range(char_arr.shape[0]):
    for col in range(char_arr.shape[1]):
        # Check if this is already apart of a region
        c = char_arr[row,col]
        if (row, col, c) in coord2region:
            continue
        else:
            # Start a region and find all connecting square
            coord2region[(row,col,c)] = region_id
            region2char[region_id] = c
            find_all_region((row,col,c), region_id, coord2region)
            region_id += 1
# coord2region

diags = [(-1,-1), (-1,1),(1,1), (1,-1)]

def diagSurroundStraight(dirs, diags):
    for i in range(len(dirs)-1):
        if ((dirs[i] == 0 and dirs[i+1] == 1 and diags[0] == 1) or
            (dirs[i] == 1 and dirs[i+1] == 2 and diags[0] == 2) or
            (dirs[i] == 2 and dirs[i+1] == 3 and diags[0] == 3) or
            (dirs[i] == 0 and dirs[i+1] == 3 and diags[0] == 0)):
            return True
    if ((dirs[0] == 0 and dirs[-1] == 1 and diags[0] == 1) or
        (dirs[0] == 1 and dirs[-1] == 2 and diags[0] == 2) or
        (dirs[0] == 2 and dirs[-1] == 3 and diags[0] == 3) or
        (dirs[0] == 0 and dirs[-1] == 3 and diags[0] == 0)):
            return True
    return False

region2coord = {}
for k,v in coord2region.items():
    region2coord.setdefault(v,[]).append((k[0],k[1]))

test_arr = np.zeros(char_arr.shape)
for k, v in coord2region.items():
    test_arr[k[0],k[1]] = v

# plt.imshow(test_arr)
# plt.xticks(np.arange(test_arr.shape[0])-0.5)
# plt.yticks(np.arange(test_arr.shape[0])-0.5)
# plt.grid()
# plt.gcf().set_size_inches((10,10))
# plt.show()

total = 0
for k,v in region2coord.items():
    area = len(v)
    num_sides = 0

    # find an outside point to start
    valid_coord = set(v)

    # If a block has 4 blocks around it, it is an interior piece
    # This could be an interior corner, check diagnols
     # 
    #X#, 4 interior corners
     #

    ## 
    #X#, 3 interior corners
     #

    ###
    #X#, 2 interior corners
     #

    ###
    #X#, 1 interior corner
    ##

    # if a block has 3 blocks around it, it is an edge
    
    #X#, 2 corners
     #

    #X#, single corner
    ##

    
    #X#, edge
    ###

    #
    #X#, single corner
    ##

      #
    #X#, edge
    ###

    # if a block has 2 blocks around it, it is outer corner OR a double edge 
    # How to tell if double edge or not
    
    #X#, double edge

      #
    #X#, edge

    # #
    #X#, edge

    #
    #X#, edge
      #

    ##
    #X, outside corner will have one diagnol

     ##
    #X, This should still be a double corner 

     ##
    #X , this is a double corner
      #

    ##
    #X , single corner
      #

    ##
    #X , single corner
    # #

    ###
    #X , single corner
    # #

    #X, an L wil not have diagnols, but should be two corners [check vertical and horizontal]
     #

    # if a block has 1 block around it, it is a super corner (two corners)

    #X, double corner

    #
    #X, double corner

    # #
    #X, double corner


    # #
    #X , double corner
      #


    for coord in v:
        num_blocks_around = 0
        dirs = []
        for d in range(4):
            if add(coord, d_unwrap[d]) in valid_coord:
                num_blocks_around += 1
                dirs.append(d)
        if num_blocks_around == 4:
            num_diags = 0
            for d in range(4):
                if add(coord, diags[d]) in valid_coord:
                    num_diags += 1
            num_sides += len(diags) - num_diags
        if num_blocks_around == 3:
            sides2add = 2
            num_diags = 0
            _diags = []
            for d in range(4):
                if add(coord, diags[d]) in valid_coord:
                    # print(dirs, d)
                    if diagSurroundStraight(dirs, [d]):
                        sides2add -= 1
            num_sides += sides2add

        if num_blocks_around == 2:
            num_diags = 0
            sides2add =  2
            _diags = []
            if not(sum(map(lambda x : 1 if x in verts else 0, dirs)) and sum(filter(lambda x : 1 if x not in verts else 0, dirs))):
                sides2add = 0
            else:
                for d in range(4):
                    if add(coord, diags[d]) in valid_coord:
                        num_diags += 1
                        if diagSurroundStraight(dirs, [d]):
                            sides2add -= 1
            num_sides += sides2add
        if num_blocks_around == 1:
            num_sides += 2
        if num_blocks_around == 0:
            num_sides += 4

    # Part 1 
    # perimeter = area * 4
    # coords = set(v)
    # for coord in coords:
    #     for d in range(4):
    #         if add(coord, d_unwrap[d]) in coords:
    #             perimeter -= 1
    total += num_sides*area
print(total)


814302
